In [70]:
import seaborn as sns
import scipy
import skimage
sns.set_style("ticks")
from pathlib import Path

In [71]:
# in this file is the url to the spreadsheet with the data
# PIGLEG_SPREADSHEET_URL=https://docs.google.com/spreadsheets/d/1G55kXxcJ0aVVCAspDUPJGkRgGoEpiLaK0snddRU-jzU/edit?usp=sharing
env_paths = list(Path(".").glob("../**/.env.dev"))
print(env_paths)
# find line with PIGLEG_SPREADSHEET_URL and give me the value after the "="
PIGLEG_SPREADSHEET_URL = [line.split("=", 1)[1].strip() for line in open(env_paths[0], "r") if line.startswith("PIGLEG_SPREADSHEET_URL")][0]
PIGLEG_SPREADSHEET_URL = PIGLEG_SPREADSHEET_URL.replace("\"", "")
PIGLEG_SPREADSHEET_URL = PIGLEG_SPREADSHEET_URL.replace("edit?usp=sharing", "export?format=csv")
# PIGLEG_SPREADSHEET_URL

[WindowsPath('../.env.dev')]


In [72]:
import pandas as pd
url = PIGLEG_SPREADSHEET_URL
df = pd.read_csv(url)

In [73]:
df.head()

,filename,report_url,email,uploaded_at,finished_at,fps,Needle holder length [m],Scissors length [m],Forceps length [m],Needle holder visibility [s],...,Scissors bbox stitch 4 visibility [%],Scissors bbox stitch 4 area presence [%],Left hand bbox stitch 1 length [pix],Left hand bbox stitch 2 length [pix],Left hand bbox stitch 3 length [pix],Left hand bbox stitch 4 length [pix],Right hand bbox stitch 1 length [pix],Right hand bbox stitch 2 length [pix],Right hand bbox stitch 3 length [pix],Right hand bbox stitch 4 length [pix]
0,pigleg_test.mp4,NaN,mjirik@gapps.zcu.cz,2022-06-09 16:13:04,2022-06-17 17:40:13,29.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,pigleg_test2.mp4,NaN,mjirik@gapps.zcu.cz,2022-04-01 20:37:52,2022-06-17 17:47:07,30.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022.03.29.-B-Voltersen-Vera-Schweinefuß-Einze...,NaN,Vera.Voltersen@med.uni-jena.de,2022-03-30 13:57:55,2022-06-17 18:07:40,30.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021.11.3-B-Volkers-Einzelknopfnaht_Schweinefu...,NaN,miriam.volkers@uni-jena.de,2021-11-04 13:19:23,2022-06-17 21:36:30,30.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Einzelknopfnaht.mp4,NaN,anke.hilger@uni-jena.de,2021-11-07 19:14:34,2022-06-17 21:36:43,30.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
# remove all records with filename NA or NaN
dfs = df[df["filename"].notna()]

# take all records with filename containing "Good", "Medium", "Bad", if the filename is the same, take the last one

dfs = dfs[dfs["filename"].str.contains("Good|Medium|Bad")].drop_duplicates(subset=["filename"], keep="last")

In [75]:
# add column "stars_auto" with 0 for "Good", 1 for "Medium", 2 for "Bad"
dfs["stars_auto"] = dfs["filename"].str.contains("Good").astype(int) * 0
dfs["stars_auto"] = dfs["filename"].str.contains("Medium").astype(int) * 1
dfs["stars_auto"] = dfs["filename"].str.contains("Bad").astype(int) * 2
# dfs["filename"].str.contains("Good|Medium|Bad")


In [76]:
len(dfs)

22

In [77]:
dfs.columns

Index(['filename', 'report_url', 'email', 'uploaded_at', 'finished_at', 'fps',
       'Needle holder length [m]', 'Scissors length [m]', 'Forceps length [m]',
       'Needle holder visibility [s]',
       ...
       'Scissors bbox stitch 4 area presence [%]',
       'Left hand bbox stitch 1 length [pix]',
       'Left hand bbox stitch 2 length [pix]',
       'Left hand bbox stitch 3 length [pix]',
       'Left hand bbox stitch 4 length [pix]',
       'Right hand bbox stitch 1 length [pix]',
       'Right hand bbox stitch 2 length [pix]',
       'Right hand bbox stitch 3 length [pix]',
       'Right hand bbox stitch 4 length [pix]', 'stars_auto'],
      dtype='object', length=376)

In [78]:
# find the columns where its values contain single "," and digits
dfs.columns[dfs.apply(lambda x: (x.dtype is str) and (x.str.contains(",\d").any()))]
cols = []
for col in dfs.columns:
    try:
        if (dfs[col].str.contains(",\d").any()):
            # convert , to . and change the column type to float
            dfs[col] = dfs[col].str.replace(",", ".").astype(float)
            cols.append(col)
    except:
        pass
cols

['Needle holder length [m]',
 'Scissors length [m]',
 'Forceps length [m]',
 'Needle holder visibility [s]',
 'Needle holder visibility [%]',
 'Forceps visibility [s]',
 'Forceps visibility [%]',
 'Scissors visibility [s]',
 'Scissors visibility [%]',
 'Needle holder area presence [%]',
 'Needle holder velocity',
 'Scissors area presence [%]',
 'Forceps area presence [%]',
 'Scissors velocity',
 'Forceps velocity',
 'pixelsize_m_by_incision_size',
 'Needle holder length [pix]',
 'Forceps length [pix]',
 'Scissors length [pix]',
 'qr_data_pix_size',
 'qr_data_qr_size',
 'qr_data_size_by_scene',
 'qr_data_pix_size_single_frame_detector_m',
 'duration_s_get_parameters_for_crop_rotate_rescale',
 'duration_s_do_crop_rotate_rescale',
 'duration_s_run_image_processing',
 'duration_s_tracking',
 'duration_s_report',
 'duration_s_stitch_ends',
 'Stitches parallelism score',
 'qr_data_scene_width_m',
 'Needle holder stitch 1 length [m]',
 'Needle holder stitch 1 visibility [s]',
 'Needle holder 

In [79]:
dfs

,filename,report_url,email,uploaded_at,finished_at,fps,Needle holder length [m],Scissors length [m],Forceps length [m],Needle holder visibility [s],...,Scissors bbox stitch 4 area presence [%],Left hand bbox stitch 1 length [pix],Left hand bbox stitch 2 length [pix],Left hand bbox stitch 3 length [pix],Left hand bbox stitch 4 length [pix],Right hand bbox stitch 1 length [pix],Right hand bbox stitch 2 length [pix],Right hand bbox stitch 3 length [pix],Right hand bbox stitch 4 length [pix],stars_auto
816,Bad-2021_11_03_B_V_lz__Ann_Sophie_Einzelknopfn...,http://suran.kky.zcu.cz:8000//uploader/web_rep...,khalatbari.sadaf@gmail.com,2022-06-20 08:33,2024-01-07 17:16,25.0,2.608826,0.230395,0.666884,101.866667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
822,Bad-2021_11_03_A_Spitze_Vanessa_Einzelknopfnah...,http://suran.kky.zcu.cz:8000//uploader/web_rep...,khalatbari.sadaf@gmail.com,2022-06-20 08:30,2024-01-07 23:39,25.0,3.909310,0.289064,0.947081,124.800000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
823,Bad-12_10_21__Loewe__Rebekka__Einzelknopfnaht.mp4,http://suran.kky.zcu.cz:8000//uploader/web_rep...,khalatbari.sadaf@gmail.com,2022-06-20 08:26,2024-01-07 23:41,25.0,3.888443,0.426266,1.262615,281.333333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
824,Bad-2021_11_03_A_Petersen_Insa_Einzelknopfnaht...,http://suran.kky.zcu.cz:8000//uploader/web_rep...,khalatbari.sadaf@gmail.com,2022-06-20 08:28,2024-01-07 23:55,25.0,3.195450,0.340322,0.414208,124.333333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
825,Good-2021_10_27_B_Etzel_Peter_Einzelknopfnaht_...,http://suran.kky.zcu.cz:8000//uploader/web_rep...,khalatbari.sadaf@gmail.com,2022-06-20 08:40,2024-01-08 00:08,25.0,3.003924,0.438534,0.700347,95.933333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
826,Bad-2021_11_3_A_Brodersen___nne_Einzelknopfnah...,http://suran.kky.zcu.cz:8000//uploader/web_rep...,khalatbari.sadaf@gmail.com,2022-06-20 08:38,2024-01-08 00:11,25.0,8.521674,0.426072,3.650445,294.266667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
827,Good-Einzelknopfnaht_Stoklossa.mp4,http://suran.kky.zcu.cz:8000//uploader/web_rep...,khalatbari.sadaf@gmail.com,2022-06-20 08:55,2024-01-08 00:46,25.0,5.030629,0.443842,1.648513,289.733333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
828,Medium-2021_10_12_B_Westermann_Lammers_Johanna...,http://suran.kky.zcu.cz:8000//uploader/web_rep...,khalatbari.sadaf@gmail.com,2022-06-20 09:00,2024-01-08 00:55,25.0,2.099404,0.196998,0.260029,69.400000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
829,Medium-2021_11_03_A_Tengler_Sophie_Einzelknopf...,http://suran.kky.zcu.cz:8000//uploader/web_rep...,khalatbari.sadaf@gmail.com,2022-06-20 09:07,2024-01-08 01:00,25.0,1.607256,0.301583,0.453664,55.666667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
830,Good-2021_11_17_A_Gramlich_Judith_Stresstest_E...,http://suran.kky.zcu.cz:8000//uploader/web_rep...,khalatbari.sadaf@gmail.com,2022-06-20 08:53,2024-01-08 01:01,25.0,12.998796,0.395285,0.197532,506.266667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [80]:
# print name of columns containing needle holder
dfs.columns[dfs.columns.str.contains("Needle holder")]

Index(['Needle holder length [m]', 'Needle holder visibility [s]',
       'Needle holder visibility [%]', 'Needle holder area presence [%]',
       'Needle holder velocity', 'Needle holder zone presence [%]',
       'Needle holder length', 'Needle holder duration', 'Needle holder unit',
       'Needle holder visibility', 'Needle holder length [pix]',
       'Needle holder stitch 1 length [m]',
       'Needle holder stitch 1 visibility [s]',
       'Needle holder stitch 1 velocity', 'Needle holder stitch 1 unit',
       'Needle holder stitch 1 visibility [%]',
       'Needle holder stitch 1 area presence [%]',
       'Needle holder bbox length [m]', 'Needle holder bbox visibility [s]',
       'Needle holder bbox velocity', 'Needle holder bbox unit',
       'Needle holder bbox visibility [%]',
       'Needle holder bbox area presence [%]',
       'Needle holder bbox stitch 1 length [m]',
       'Needle holder bbox stitch 1 visibility [s]',
       'Needle holder bbox stitch 1 velocity',
 

In [81]:
dfs[["Needle holder stitch 1 length [m]", "Needle holder length [m]"]]

,Needle holder stitch 1 length [m],Needle holder length [m]
816,NaN,2.608826
822,NaN,3.909310
823,NaN,3.888443
824,NaN,3.195450
825,NaN,3.003924
826,3.834157,8.521674
827,2.105825,5.030629
828,NaN,2.099404
829,NaN,1.607256
830,NaN,12.998796


In [82]:
# If column containing " stitch 1" in its name has empty values, fill them with  the values from the column with the same name but withuou " stitch 1"
for col in dfs.columns[dfs.columns.str.contains(" stitch 1")]:
    dfs[col] = dfs[col].fillna(dfs[col.replace(" stitch 1", "")])

In [83]:
dfs[["Needle holder stitch 1 length [m]", "Needle holder length [m]"]]


,Needle holder stitch 1 length [m],Needle holder length [m]
816,2.608826,2.608826
822,3.909310,3.909310
823,3.888443,3.888443
824,3.195450,3.195450
825,3.003924,3.003924
826,3.834157,8.521674
827,2.105825,5.030629
828,2.099404,2.099404
829,1.607256,1.607256
830,12.998796,12.998796


In [84]:
# find corelation between stars_auto and other numerical columns and sort them
dfs.corr()["stars_auto"].sort_values()



C:\Users\Jirik\AppData\Local\Temp\ipykernel_1412\457172700.py:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  dfs.corr()["stars_auto"].sort_values()


Forceps bbox visibility [%]             -0.479464
Stitches parallelism score              -0.330578
Scissors visibility [%]                 -0.325639
n_stitches_by_user                      -0.319676
Forceps bbox visibility [s]             -0.254723
                                           ...   
Left hand bbox stitch 4 length [pix]          NaN
Right hand bbox stitch 1 length [pix]         NaN
Right hand bbox stitch 2 length [pix]         NaN
Right hand bbox stitch 3 length [pix]         NaN
Right hand bbox stitch 4 length [pix]         NaN
Name: stars_auto, Length: 270, dtype: float64

In [89]:
# try to find the best columns to predict stars_auto
corr = dfs.corr()["stars_auto"].sort_values(ascending=False).head(10)
corr


C:\Users\Jirik\AppData\Local\Temp\ipykernel_1412\1885081676.py:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr = dfs.corr()["stars_auto"].sort_values(ascending=False).head(10)


stars_auto                                       1.000000
Forceps stitch 1 length [m]                      0.392658
Forceps stitch 1 area presence [%]               0.344862
qr_data_scene_width_m                            0.336219
qr_data_pix_size                                 0.336219
Needle holder area presence [%]                  0.319609
Forceps area presence [%]                        0.304575
orig frame_height                                0.292419
Needle holder bbox stitch 1 area presence [%]    0.288653
Forceps length [m]                               0.277674
Name: stars_auto, dtype: float64

In [90]:
import sklearn
# train a model to predict stars_auto
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
# import SVM
from sklearn.svm import SVR
#train a model to predict stars_auto

columns = list(corr.index)
columns.append("filename")

dfsx = dfs[columns]
print(len(dfsx))
dfsx = dfsx.dropna()
print(len(dfsx))
X = dfsx[corr.index].drop("stars_auto", axis=1)
y = dfsx["stars_auto"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = SVR()
model.fit(X_train, y_train)
model.score(X_test, y_test)
# 
# X

22
20


-0.8008979668108016

In [91]:
y_test

816    2
841    0
839    0
822    2
Name: stars_auto, dtype: int32

In [92]:
y_pred = model.predict(X_test)
y_pred

array([0.10882825, 0.09457567, 0.09590648, 0.10028458])

In [94]:
model.predict(X_train)


array([0.10021047, 0.10583454, 0.09953493, 0.10522731, 0.10041116,
       0.05032754, 0.10958229, 0.10592193, 0.11030752, 0.10254129,
       0.09964285, 0.10444273, 0.10041114, 0.09998731, 0.10081808,
       0.08631827])

In [95]:
y_train

829    0
826    2
833    0
824    2
842    0
840    0
835    2
823    2
830    0
843    0
825    0
834    2
828    0
832    0
838    2
827    0
Name: stars_auto, dtype: int32

In [96]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(5, 2), random_state=1)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.5

In [99]:
clf.predict(X_train)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [98]:
y_test

816    2
841    0
839    0
822    2
Name: stars_auto, dtype: int32